In [1]:
from src.AlphabetPermutation import AlphabetPermutation
from src.Probability import ProbabilityMatrix
from string import ascii_lowercase
import numpy as np

In [2]:
alphabet = list(ascii_lowercase)
alphabet.append(' ')

# Initialize an instance of permutation generator class
permutation_generator = AlphabetPermutation(alphabet)

# Obtain the key: in a cipher the key is the alphabet used by the cipher
key = permutation_generator.permutation.values()

In [3]:
message = "Hello MCMC let us see how well you decrypt messages"

In [4]:
encrypted_message = permutation_generator.permute_string(message)
encrypted_message

'nfssy hghg sfc ti iff nyv vfss ayt lfgqakc hfiimofi'

In [5]:
with open("texts/moby_dick.txt", "r") as file:
    text = file.read()
alphabet = list("abcdefghijklmnopqrstuvwxyz ")

In [6]:
pm = ProbabilityMatrix(text=text, alphabet=alphabet)
unknown_chars = pm.unknown_chars()
pm.preprocess_text(unknown_chars=unknown_chars)

pm.compute_matrix_spaces()
log_matrix = pm.compute_log_matrix()

In [10]:
len(pm.matrix[0])

27

In [27]:
def compute_likelihood(encrypted_message, log_transition_matrix):
    # Add whitespace before the message
    encrypted_message = " " + encrypted_message

    log_lik = 0
    for i in range(len(encrypted_message)-1):
        # Convert letters to corresponding entry in the alphabet (a = 0, b = 2, ..., ' ' = 26)
        first_letter = ord(encrypted_message[i])-97
        second_letter = ord(encrypted_message[i+1])-97

        if(first_letter == 32 - 97): #whitespace correpsonds to 32
            first_letter = 26
        if(second_letter == 32 - 97):
            second_letter = 26

        log_lik = log_lik + log_transition_matrix[first_letter, second_letter]
    
    return log_lik

In [29]:
print("log likelihood for encrypted message:", compute_likelihood(encrypted_message, log_matrix))
print("log likelihood for original message:", compute_likelihood(message.lower(), log_matrix))

log likelihood for encrypted message: -440.09383163981823
log likelihood for original message: -359.21261536436083
